In [1]:
import pandas as pd

In [11]:
path = 'C:/Users/UserK/Desktop/Ranee/data/신용예측프로젝트/프로젝트 데이터/'

In [18]:
df1 = pd.read_csv(path + '경영진정보_train.csv')
df2 = pd.read_csv(path +'국가R&D과제_train.csv')
df3 = pd.read_csv(path +'상표권정보_train.csv')
df4 = pd.read_csv(path +'온라인뉴스_train.csv')
df5 = pd.read_csv(path +'국가R&D과제_train.csv')
df6 = pd.read_csv(path +'국가R&D과제_train.csv')

C:\Users\UserK\AppData\Local\Temp\ipykernel_8412\61049774.py:2: DtypeWarning: Columns (42,44,46,48,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(path +'국가R&D과제_train.csv')
C:\Users\UserK\AppData\Local\Temp\ipykernel_8412\61049774.py:4: DtypeWarning: Columns (42,44,46,48,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(path +'국가R&D과제_train.csv')
C:\Users\UserK\AppData\Local\Temp\ipykernel_8412\61049774.py:5: DtypeWarning: Columns (42,44,46,48,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv(path +'국가R&D과제_train.csv')
C:\Users\UserK\AppData\Local\Temp\ipykernel_8412\61049774.py:6: DtypeWarning: Columns (42,44,46,48,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv(path +'국가R&D과제_train.csv')


In [20]:
df3

,Unnamed: 0,BusinessNum,agentName,applicantName,applicationDate,applicationNumber,classificationCode,registrationNumber,title
0,0,1051505526,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2118605035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2298104710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4818104219,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1178100918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15121,15121,1248600665,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15122,15122,2068607541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15123,15123,8778607222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15124,15124,1448101319,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df4 = pd.read_csv(path +'온라인뉴스_train.csv')

In [32]:
df4['NewsTitle'].dropna()

18                   반도체 상장기업 기업평판 11월 우수기업 발표
19                   반도체 상장기업 기업평판 11월 우수기업 발표
27       방통위, 인수위에 2명 파견…경제2분과·과학기술교육분과 ICT 전무
28       방통위, 인수위에 2명 파견…경제2분과·과학기술교육분과 ICT 전무
29       방통위, 인수위에 2명 파견…경제2분과·과학기술교육분과 ICT 전무
                         ...                  
46434               코이드, 전화영어 '스피크 114' 서비스 개시
46435               [동아리가 좋아요] 코이드 포켓볼동호회 ‘한큐’
46436                     [월요기획-전화영어]주요서비스-코이드
46443                      (주)뉴로메카 코스닥시장 상장기념식
46444       [특징주] 뉴로메카제이아이테크 상장 첫날 희비 엇갈린 새내기주
Name: NewsTitle, Length: 9225, dtype: object

In [33]:
df5 = pd.read_csv(path +'신용등급정보_train.csv')

In [34]:
df5.head(5)

,Unnamed: 0,BusinessNum,cmpCd,cmpNm,cmpScl,criGrd,accDate,grdDesc
0,0,1051505526,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2118605035,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2298104710,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4818104219,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1178100918,1673782.0,엠피엠지,중소기업,A2,2022-07-19,"양호한 단기거래 신용능력을 보유하여, 채무불이행으로 인한 위험 가능성이 낮으며, 현..."


In [35]:
df1.head(5)

,Unnamed: 0,BusinessNum,cmpCd,mgrNm,pstnCd,pstnCdNm,chrgTaskNm,rgstStat,rgstStatNm,mgrEnm,brthDate,zip,adr,dtlAdr,eduCont,crrCont
0,0,1051505526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2118605035,1831017.0,램춘센데이비드,31.0,사장,NaN,1.0,등기,Lamchoonsen David,1952.0,NaN,NaN,NaN,NaN,NaN
2,2,2298104710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4818104219,8141947.0,김재문,30.0,대표이사사장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4818104219,8141947.0,김재문,30.0,대표이사사장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df6 = pd.read_csv(path +'기업정보요약_train.csv')

In [38]:
df6.head(5)

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm
0,0,1131201292,20210903,에**,김**,02 26333232,NaN,7295.0,서울특별시 영등포구 문래로20길,"60, 5층 505호 (문래동3가,메가벤처타워)",일반,중소기업,일반기업,42321.0,일반 통신 공사업
1,1,6058609398,20220308,두원건설***,이**,043 283 4567,043 292 2002,28536.0,충청북도 청주시 청원구 중앙로 140,4층 (우암동),일반,중소기업,일반기업,41100.0,건물 건설업
2,2,1138601802,20210903,(주)드림아이***,유**,070 77666664,02 62806509,8510.0,서울 금천구 벚꽃로 298,"903호,1312호 (가산동,대륭포스트타워6차)",외감,중소기업,일반기업,62010.0,컴퓨터 프로그래밍 서비스업
3,3,7048608706,20211201,(주)드림오브***,신**,02-854-8000,050-4199-5335,3900.0,서울 마포구 마포나루길 332,"선착장 (상암동,상암나루)",일반,중소기업,일반기업,71531.0,경영컨설팅업
4,4,5088103009,20211130,(주)세***,권**,054 853 0006,054 8539929,36660.0,경북 안동시 경북대로,"389, 101호 지하",일반,중소기업,일반기업,41220.0,토목시설물 건설업
